# EPMT Query API

This workbook will illustrate the usage of the EPMT Query API. It assumes you have `EPMT`
installed.


## Table of Contents

 * [Import data for the study](#import-data)
 * [Import module](#import-module)
 * [Getting documentation](#getting-docs)
 * [Job Query](#job-query)
   * [output format and converting between formats](#output-formats)
   * [working with ORM objects (ADVANCED TOPIC)](#orm-objects)
   * [job tags](#job-tags)
   * [ordering and filtering jobs](#jobs-order-filter)
   * [failed jobs](#failed-jobs)
   * [process sums (ADVANCED TOPIC)](#proc-sums-field)
 * [Process Query](#process-query)
   * [process tags](#process-tags)
     * [unique process tags in job (ADVANCED TOPIC)](#job-proc-tags)
   * [filter and ordering](#filter-processes)
   * [thread metrics aggregation (ADVANCED TOPIC)](#thread-metrics-aggregation)
 * [Operations](#ops)
   * [select op processes](#select-op-procs)
   * [operation queries (ADVANCED TOPIC)](#op-queries)
     * [aggregate cpu time for operation](#cumulative-cpu-time-for-op)
     * [select top executables by cpu time](#op-top-exe)
   * [aggregating operation metrics](#op-metrics)
   * [data-movement v. useful work](#dm-ops)
   * [op_metrics grouped by tag](#group-by-tag)
   * [cpu-time v. duration](#cpu-time-v-duration)
 * [Thread Query](#thread-query)
 * [Logging and SQL Debug](#logging-debug)
 * [Useful Queries](#useful-queries)
   * [process tree walk](#process-tree-walk)
   * [failed processes](#failed-procs)
   * [all process tags for job](#job-proc-tags)
   * [root process](#root-process)
   * [timeline](#timeline)
 * [Useful Attributes of Job/Process/Threads](#useful-attributes)
 * [Case Study - Linux Kernel](#linux-kernel)
   * [Timeline (ADVANCED TOPIC)](#timeline)


## <a name="import-data">Import the data for this study</a>

This workbook relies on importing the following data. We use an sqlite database 
in this study, but you can use another database such as `postgresql`.
See the `settings` folder to pick up a template of your choice and edit it
if needed. The commands below will need to be run in the top-level directory of your
EPMT installation.

```
# pick the database settings file of your choice
$ cp settings/settings_sqlite_localfile.py settings.py

# now import the data
$ ./epmt -v submit $(cat <<EOF
sample/ppr-batch/1864/629337.tgz
sample/ppr-batch/1854/625172.tgz
sample/ppr-batch/1879/680181.tgz
sample/ppr-batch/1859/627922.tgz
sample/ppr-batch/1899/696127.tgz
sample/ppr-batch/1869/633144.tgz
sample/ppr-batch/1874/676007.tgz
sample/ppr-batch/1884/685016.tgz
sample/ppr-batch/1889/692544.tgz
sample/ppr-batch/1904/802954.tgz
sample/ppr-batch/1894/693147.tgz
sample/ppr-batch/1909/804285.tgz
EOF
)
```

<a name="import-module"></a>

In [1]:
# import the query api module
import epmt_query as eq

INFO:epmt_job:Binding to DB: {'provider': 'sqlite', 'create_db': True, 'filename': 'database.sqlite'}
INFO:epmt_job:Generating mapping from schema...


The API has a few queries -- `get_jobs`, `get_procs` and `get_thread_metrics` -- that you will be using frequently.

Each of these operate at distinct levels: job, process and threads.

### <a name="getting-docs">Getting to the docs</a>

The module functions have embedded documentation in the form of docstrings. You can access it, 
as you would do for any Python module/function:

To get help for all functions in the module, do `help(<module-name)`:
```
help(eq)
```

To get documentation for a specific function, do something like:
```
help(eq.get_jobs)
```

### <a name="job-query">Job Query</a>

The job query usually takes a `tag` and returns a collection of jobs in the format specified by `fmt`.
The returned list can be pruned and/or ordered using `fltr`, `limit` and `order`.

You can also pass in one or more jobs as a `jobs` parameter, most often for format conversion.

Let's get started!

In [2]:
# let's get jobs, we use the job tag to select the jobs
jobs = eq.get_jobs(tag='exp_name:ESM4_historical_D151;exp_component:ocean_month_rho2_1x1deg',fmt='terse')
jobs

['804285',
 '693147',
 '802954',
 '692544',
 '685016',
 '676007',
 '633144',
 '696127',
 '627922',
 '680181',
 '625172',
 '629337']

<a name="output-formats"></a>`fmt` can take one of the following values:
 * `terse` -- this returns a list of job ids
 * `pandas` -- this returns a pandas dataframe
 * `dict` -- for a list of python dictionaries
 * `orm` -- ORM object for maximum flexibility and speediest queries.

In [3]:
# above we got a list of job ids. sometimes we want to see more details
# than just the job id. We can use `conv_jobs` to convert between formats
jobs_df = eq.conv_jobs(jobs, fmt='pandas')
display(jobs_df.columns.values)
jobs_df

array(['PERF_COUNT_SW_CPU_CLOCK', 'account', 'all_proc_tags',
       'cancelled_write_bytes', 'cpu_time', 'created_at',
       'delayacct_blkio_time', 'duration', 'end', 'env_changes_dict',
       'env_dict', 'exitcode', 'guest_time', 'inblock', 'info_dict',
       'invol_ctxsw', 'jobid', 'jobname', 'jobscriptname', 'majflt',
       'minflt', 'num_procs', 'num_threads', 'outblock', 'processor',
       'queue', 'rchar', 'rdtsc_duration', 'read_bytes', 'rssmax',
       'sessionid', 'start', 'submit', 'syscr', 'syscw', 'systemtime',
       'tags', 'time_oncpu', 'time_waiting', 'timeslices', 'updated_at',
       'user', 'user+system', 'usertime', 'vol_ctxsw', 'wchar',
       'write_bytes'], dtype=object)

,PERF_COUNT_SW_CPU_CLOCK,account,all_proc_tags,cancelled_write_bytes,cpu_time,created_at,delayacct_blkio_time,duration,end,env_changes_dict,...,time_oncpu,time_waiting,timeslices,updated_at,user,user+system,usertime,vol_ctxsw,wchar,write_bytes
0,866184046169,None,"[{'op': 'cp', 'op_sequence': '119', 'op_instan...",2448543744,959968389,2019-08-05 07:08:37.993287,0,12630660818,2019-06-09 22:23:53.234877,{},...,967130737639,48105200287,3175850,2019-08-05 07:08:37.993290,Jeffrey.Durachta,959968389,769390290,3132299,138638409056,134643470336
1,649725614194,None,"[{'op': 'cp', 'op_sequence': '66', 'op_instanc...",3044032512,679701225,2019-08-05 07:09:26.941451,0,6532173945,2019-06-10 08:12:06.562689,{},...,681730630413,13802065001,824685,2019-08-05 07:09:26.941454,Jeffrey.Durachta,679701225,428874880,809138,74236894004,72541306880
2,576733745205,None,"[{'op': 'cp', 'op_sequence': '66', 'op_instanc...",1998835712,623964730,2019-08-05 07:08:27.874242,0,6696039124,2019-06-10 11:50:58.082917,{},...,625961524504,19476877777,805888,2019-08-05 07:08:27.874245,Jeffrey.Durachta,623964730,478902294,792701,74236893737,70252703744
3,582075035986,None,"[{'op': 'cp', 'op_sequence': '66', 'op_instanc...",1385582592,621768978,2019-08-05 07:09:47.063038,0,6625637678,2019-06-10 18:39:32.439890,{},...,623723072793,24198886180,826722,2019-08-05 07:09:47.063041,Jeffrey.Durachta,621768978,467152088,793749,74236867345,70780125184
4,605349533801,None,"[{'op': 'cp', 'op_sequence': '66', 'op_instanc...",1465577472,640906121,2019-08-05 07:09:57.237769,0,10080732883,2019-06-14 11:18:38.154111,{},...,642678934515,23712986693,849400,2019-08-05 07:09:57.237773,Jeffrey.Durachta,640906121,450158690,832964,74236894547,73864601600
5,522980958427,None,"[{'op': 'cp', 'op_sequence': '66', 'op_instanc...",1998835712,571561896,2019-08-05 07:09:16.809929,0,6009933600,2019-06-14 18:14:24.986076,{},...,573565404518,32443024755,818718,2019-08-05 07:09:16.809932,Jeffrey.Durachta,571561896,434850361,793333,74236809886,70259195904
6,403054040862,None,"[{'op': 'cp', 'op_sequence': '66', 'op_instanc...",3392450560,427082965,2019-08-05 07:10:07.340292,0,7005618511,2019-06-15 09:49:24.210549,{},...,429280375226,11304090572,812300,2019-08-05 07:10:07.340295,Jeffrey.Durachta,427082965,332821891,799028,74236867987,70246367232
7,557685977561,None,"[{'op': 'cp', 'op_sequence': '66', 'op_instanc...",3392417792,593701277,2019-08-05 07:10:18.647149,0,709300857,2019-06-16 14:06:18.129747,{},...,595771270622,18759054582,797027,2019-08-05 07:10:18.647152,Jeffrey.Durachta,593701277,457078582,783079,74236883941,70606073856
8,553117186630,None,"[{'op': 'cp', 'op_sequence': '66', 'op_instanc...",1998848000,594222175,2019-08-05 07:10:38.903957,0,3340305357,2019-06-16 17:16:11.907347,{},...,596382718874,21984544439,801396,2019-08-05 07:10:38.903961,Jeffrey.Durachta,594222175,452663282,783373,74236938446,70251761664
9,574686010894,None,"[{'op': 'cp', 'op_sequence': '66', 'op_instanc...",2347225088,607235263,2019-08-05 07:09:36.998433,0,3676905115,2019-06-17 07:22:16.747572,{},...,609161751217,17253128153,813225,2019-08-05 07:09:36.998437,Jeffrey.Durachta,607235263,468679853,797766,74236837177,70476115968


In [4]:
# if you prefer dealing with python lists and dictionaries,
# you can set fmt='dict'. Here we get a list of dictionaries
eq.get_jobs(jobs = jobs, fmt='dict')

[{'PERF_COUNT_SW_CPU_CLOCK': 866184046169,
  'account': None,
  'all_proc_tags': [{'op': 'cp', 'op_instance': '1', 'op_sequence': '119'},
   {'op': 'cp', 'op_instance': '1', 'op_sequence': '122'},
   {'op': 'cp', 'op_instance': '1', 'op_sequence': '123'},
   {'op': 'cp', 'op_instance': '11', 'op_sequence': '167'},
   {'op': 'cp', 'op_instance': '15', 'op_sequence': '180'},
   {'op': 'cp', 'op_instance': '3', 'op_sequence': '131'},
   {'op': 'cp', 'op_instance': '5', 'op_sequence': '140'},
   {'op': 'cp', 'op_instance': '7', 'op_sequence': '149'},
   {'op': 'cp', 'op_instance': '9', 'op_sequence': '158'},
   {'op': 'dmput', 'op_instance': '1', 'op_sequence': '126'},
   {'op': 'dmput', 'op_instance': '2', 'op_sequence': '190'},
   {'op': 'fregrid', 'op_instance': '1', 'op_sequence': '117'},
   {'op': 'fregrid', 'op_instance': '1', 'op_sequence': '121'},
   {'op': 'fregrid', 'op_instance': '2', 'op_sequence': '132'},
   {'op': 'fregrid', 'op_instance': '3', 'op_sequence': '141'},
   {'op'

<a name="orm-objects"></a>
There is a very useful format called ORM, this optimizes queries
and it lets you get the underlying Job (or Process) object directly

In [5]:
jobs_orm = eq.get_jobs(jobs, fmt='orm')
jobs_orm.count(), type(jobs_orm)

(12, pony.orm.core.Query)

`jobs_orm` above is a Pony `Query` object. The `Query` object can be iterated
over (like a Python list). You can convert it to a list by using the slice
operator -- `[:]`.

The ORM format is powerful as it minimizes the number of SQL queries.
Let's see this in action. To do so, we need to enable SQL debug. This
first requires setting logging to INFO level or higher. <a name="logging-debug"></a>

Now, we will run a query first using a format other than ORM, say 'terse', 
and then using the 'orm' format. You will notice that in ORM format SQL queries are
"lazy-evaluated", leading to fewer queries. It's only for the ORM type of result that 
you can use functions like: `sum`, `count`, `avg`, etc. For other objects such 
as a list or pandas dataframe, you would use functions like `len`.

In [6]:
eq.set_logging(1)
eq.set_sql_debug(True)

In [7]:
jobs = eq.get_jobs(tag='exp_component:ocean_month_rho2_1x1deg',fmt='terse')

INFO:pony.orm.sql:SELECT "j"."start", "j"."end", "j"."duration", "j"."proc_sums", "j"."created_at", "j"."updated_at", "j"."info_dict", "j"."env_dict", "j"."env_changes_dict", "j"."submit", "j"."jobid", "j"."jobname", "j"."jobscriptname", "j"."sessionid", "j"."exitcode", "j"."user", "j"."tags", "j"."account", "j"."queue", "j"."cpu_time"
FROM "Job" "j"
WHERE CAST(json_extract("j"."tags", ?) as text) = ?
ORDER BY "j"."created_at" DESC
LIMIT 20
INFO:pony.orm:RELEASE CONNECTION


In [8]:
jobs_orm =  eq.get_jobs(tag='exp_component:ocean_month_rho2_1x1deg',fmt='orm')

In [9]:
# Notice for the ORM, the query hasn't been run yet. Now, let's do a count
# of the jobs. You will see that rather than loading the jobs from the DB,
# only a COUNT sql query is run
jobs_orm.count()

INFO:pony.orm:GET CONNECTION FROM THE LOCAL POOL
INFO:pony.orm:SWITCH TO AUTOCOMMIT MODE
INFO:pony.orm.sql:SELECT COUNT(*)
FROM "Job" "j"
WHERE CAST(json_extract("j"."tags", ?) as text) = ?


12

In [10]:
# now let's remove the logging and sql debug to avoid cluttering the output
eq.set_sql_debug(False)
eq.set_logging(0)

#### <a name="job-tags">Job Tags</a>

Each job has a `tags` field that is set during import time. The job tag is a stored
as dictionary of key/value pairs. The most common use of the job tag is for selecting
jobs. You can specify the tag either as a dictionary or as a string, with each key/value
pair separated by semicolons. All the key/value pairs must match for a job to be considered
a match.

In [11]:
jobs_190900101 = eq.get_jobs(tag='exp_name:ESM4_historical_D151;exp_component:ocean_month_rho2_1x1deg;exp_time:19090101', fmt='orm')

In [12]:
for j in jobs_190900101:
    print(j.jobid, j.tags)

804285 {'exp_component': 'ocean_month_rho2_1x1deg', 'atm_res': 'c96l49', 'script_name': 'ESM4_historical_D151_ocean_month_rho2_1x1deg_19090101', 'ocn_res': '0.5l75', 'exp_time': '19090101', 'exp_name': 'ESM4_historical_D151'}


In [13]:
# *** ADVANCED TOPIC ***
# now let's see the job tags for each of the jobs in the ORM `Query` object
eq.select((j.jobid, j.tags) for j in jobs_orm)[:]

[('804285', {'exp_component': 'ocean_month_rho2_1x1deg', 'atm_res': 'c96l49', 'script_name': 'ESM4_historical_D151_ocean_month_rho2_1x1deg_19090101', 'ocn_res': '0.5l75', 'exp_time': '19090101', 'exp_name': 'ESM4_historical_D151'}), ('693147', {'exp_component': 'ocean_month_rho2_1x1deg', 'atm_res': 'c96l49', 'script_name': 'ESM4_historical_D151_ocean_month_rho2_1x1deg_18940101', 'ocn_res': '0.5l75', 'exp_time': '18940101', 'exp_name': 'ESM4_historical_D151'}), ('802954', {'exp_component': 'ocean_month_rho2_1x1deg', 'atm_res': 'c96l49', 'script_name': 'ESM4_historical_D151_ocean_month_rho2_1x1deg_19040101', 'ocn_res': '0.5l75', 'exp_time': '19040101', 'exp_name': 'ESM4_historical_D151'}), ('692544', {'exp_component': 'ocean_month_rho2_1x1deg', 'atm_res': 'c96l49', 'script_name': 'ESM4_historical_D151_ocean_month_rho2_1x1deg_18890101', 'ocn_res': '0.5l75', 'exp_time': '18890101', 'exp_name': 'ESM4_historical_D151'}), ('685016', {'exp_component': 'ocean_month_rho2_1x1deg', 'atm_res': 'c96

#### <a name="jobs-order-filter">Ordering and Filtering Jobs</a>

You can use the `order`, `limit`, and `fltr` option with `get_jobs` to sort and filter the job list.
It is advisable to use `limit` when possible, as it sends a `LIMIT` option to the SQL query
and saves database load time.

In [14]:
# some other useful queries might be for instance to order the jobs
# by duration, and getting the top 5
df = eq.get_jobs(jobs_orm, order='desc(j.duration)', limit=5, fmt="pandas")
df[['jobid', 'tags', 'duration', 'exitcode']]

,jobid,tags,duration,exitcode
0,625172,"{'exp_component': 'ocean_month_rho2_1x1deg', '...",12630660818,0
1,676007,"{'exp_component': 'ocean_month_rho2_1x1deg', '...",10080732883,0
2,685016,"{'exp_component': 'ocean_month_rho2_1x1deg', '...",7005618511,0
3,629337,"{'exp_component': 'ocean_month_rho2_1x1deg', '...",6696039124,0
4,633144,"{'exp_component': 'ocean_month_rho2_1x1deg', '...",6625637678,0


<a name="failed-jobs"></a>Let's figure out which if any jobs failed.

In [15]:
eq.get_jobs(jobs_orm, fltr='j.exitcode != 0', fmt='terse')

[]

#### <a name="proc-sums-field">Aggregation across job processes (ADVANCED TOPIC)</a>

Each job object has a `proc_sums` field that aggregates data across the 
processes of the job. The field itself is a dictionary of key/value pairs.
This field is an attribute in the Job object, and when converting from `orm` 
to the other formats, the underlying key/value pairs of the dictionary are made available 
as top-level fields of the `dict` or `pandas` dataframe. `proc_sums` represents aggregates across
the processes of a job:

In [16]:
j = jobs_orm.first()
j.proc_sums.keys()

dict_keys(['rssmax', 'guest_time', 'timeslices', 'rdtsc_duration', 'syscr', 'usertime', 'all_proc_tags', 'read_bytes', 'PERF_COUNT_SW_CPU_CLOCK', 'num_procs', 'majflt', 'syscw', 'invol_ctxsw', 'minflt', 'wchar', 'outblock', 'cancelled_write_bytes', 'time_waiting', 'systemtime', 'delayacct_blkio_time', 'write_bytes', 'inblock', 'vol_ctxsw', 'time_oncpu', 'processor', 'user+system', 'rchar', 'num_threads'])

Now, the fields shown above become available in other formats (`dict` and `pandas`) as top-level fields, while the `proc_sums`
field itself is masked.

In [17]:
j_df = eq.get_jobs(j, fmt='pandas')
j_df.columns.values

array(['PERF_COUNT_SW_CPU_CLOCK', 'account', 'all_proc_tags',
       'cancelled_write_bytes', 'cpu_time', 'created_at',
       'delayacct_blkio_time', 'duration', 'end', 'env_changes_dict',
       'env_dict', 'exitcode', 'guest_time', 'inblock', 'info_dict',
       'invol_ctxsw', 'jobid', 'jobname', 'jobscriptname', 'majflt',
       'minflt', 'num_procs', 'num_threads', 'outblock', 'processor',
       'queue', 'rchar', 'rdtsc_duration', 'read_bytes', 'rssmax',
       'sessionid', 'start', 'submit', 'syscr', 'syscw', 'systemtime',
       'tags', 'time_oncpu', 'time_waiting', 'timeslices', 'updated_at',
       'user', 'user+system', 'usertime', 'vol_ctxsw', 'wchar',
       'write_bytes'], dtype=object)

### <a name="process-query">Process Query</a>

A process query returns a collection of one or more processes. The query is
passed a `jobs` parameter to restrict the process set to those belong to a
specified set of `jobs`. 

Like the job query, the process query can take `tag`, `fmt`, 
`fltr`, `order` and `limit` to filter and format the output. `order` and `limit` become
particularly important in process queries as each job can have thousands of processes,
and that takes time to load from the database. In the same vein, using `fmt=orm` is a good
idea, in process queries.

In [18]:
# If you want to get the processes belonging to a job
# here each row in the pandas dataframe contains one job process
# again, you can use the 'terse' fmt option to get just the list of database ids of the processes
eq.get_procs(['629337'], fmt='pandas')[:10]

,PERF_COUNT_SW_CPU_CLOCK,args,cancelled_write_bytes,cpu_time,created_at,delayacct_blkio_time,duration,end,exename,exitcode,...,time_oncpu,time_waiting,timeslices,updated_at,user,user+system,usertime,vol_ctxsw,wchar,write_bytes
0,755006778,-f /home/Jeffrey.Durachta/ESM4/DECK/ESM4_histo...,4096,807876,2019-08-05 07:08:36.441276,0,6.696012e+09,2019-06-10 15:50:58.073534,tcsh,0,...,808668773,427323246,3804,2019-08-05 07:08:36.441278,Jeffrey.Durachta,807876,454930,3799,82957,5451776
1,116082,-f /home/Jeffrey.Durachta/ESM4/DECK/ESM4_histo...,0,0,2019-08-05 07:08:28.056156,0,1.200000e+02,2019-06-10 13:59:22.064416,tcsh,0,...,1966005,24572,1,2019-08-05 07:08:28.056160,Jeffrey.Durachta,0,0,0,0,0
2,177971,-p /vftmp/Jeffrey.Durachta/job629337/ESM4/DECK...,0,3998,2019-08-05 07:08:28.058969,0,1.820000e+02,2019-06-10 13:59:22.074184,mkdir,0,...,4907282,42356,5,2019-08-05 07:08:28.058972,Jeffrey.Durachta,3998,1999,4,0,0
3,5032465,tcsh use -a /home/fms/local/modulefiles,0,16996,2019-08-05 07:08:28.061391,0,5.215000e+03,2019-06-10 13:59:22.137614,modulecmd,0,...,17479352,101721,10,2019-08-05 07:08:28.061394,Jeffrey.Durachta,16996,8998,9,80,4096
4,97976,0 = 0,0,10997,2019-08-05 07:08:28.063768,0,1.060000e+02,2019-06-10 13:59:22.155449,test,0,...,11612639,67333,6,2019-08-05 07:08:28.063771,Jeffrey.Durachta,10997,7998,5,11,4096
5,3840149,tcsh purge,0,14997,2019-08-05 07:08:28.066099,0,4.037000e+03,2019-06-10 13:59:22.178281,modulecmd,0,...,15719947,97660,12,2019-08-05 07:08:28.066101,Jeffrey.Durachta,14997,5999,10,51,4096
6,88208,0 = 0,0,10997,2019-08-05 07:08:28.068462,0,9.600000e+01,2019-06-10 13:59:22.199641,test,0,...,11355247,95352,7,2019-08-05 07:08:28.068465,Jeffrey.Durachta,10997,6998,5,11,4096
7,199420281,tcsh load fre/bronx-15,0,213967,2019-08-05 07:08:28.103976,0,6.472980e+05,2019-06-10 13:59:22.866366,modulecmd,0,...,214669612,1252984,215,2019-08-05 07:08:28.103979,Jeffrey.Durachta,213967,181972,207,4882,4096
8,47913625,-T -e use Net::Domain(hostdomain); print hostd...,0,59990,2019-08-05 07:08:28.070804,0,4.819800e+04,2019-06-10 13:59:22.298420,perl,0,...,60730333,184677,9,2019-08-05 07:08:28.070807,Jeffrey.Durachta,59990,49992,6,25,0
9,12715756,"-e use English; printf ""%vd"", $PERL_VERSION",0,24995,2019-08-05 07:08:28.073149,0,1.274700e+04,2019-06-10 13:59:22.328123,perl,0,...,25407862,119673,7,2019-08-05 07:08:28.073152,Jeffrey.Durachta,24995,17997,4,6,0


You could also pass in more than one job, in which case the returned processes
would be a superset of those across the jobs list. Here we use the `orm` format
to speed the query since we just want a `count` of processes.

In [19]:
procs = eq.get_procs(['629337', '625172'], fmt='orm')
procs.count()

15943

#### <a name="process-tags">Process Tags</a>

Each process saves a dictionary of key/value pairs, such as:
`{'op': "ncatted", 'op_instance': 12, 'op_sequence': 159}`

The process tag is commonly used to filter processes that constitute an **operation** using the `tag` option.

In [20]:
# below we get the processes in an operation that is identified by 'op_sequence=66'
op = eq.get_procs(jobs, tag='op:cp;op_instance:11;op_sequence:66', fmt='pandas')
len(op)

1914

##### <a name="job-proc-tags">Unique process tags in a job (ADVANCED TOPIC)</a>

For a job we can determine the unique set of process tags</a> across all its processes using the
`job_proc_tags` API call.

In [21]:
# suppose you want to figure out the unique set of operations
# across all the jobs of interest. We would pass in our list of
# jobs
eq.job_proc_tags(jobs_orm)

[{'op': 'cp', 'op_instance': '11', 'op_sequence': '66'},
 {'op': 'cp', 'op_instance': '15', 'op_sequence': '79'},
 {'op': 'cp', 'op_instance': '3', 'op_sequence': '30'},
 {'op': 'cp', 'op_instance': '5', 'op_sequence': '39'},
 {'op': 'cp', 'op_instance': '7', 'op_sequence': '48'},
 {'op': 'cp', 'op_instance': '9', 'op_sequence': '57'},
 {'op': 'dmput', 'op_instance': '2', 'op_sequence': '89'},
 {'op': 'fregrid', 'op_instance': '2', 'op_sequence': '31'},
 {'op': 'fregrid', 'op_instance': '3', 'op_sequence': '40'},
 {'op': 'fregrid', 'op_instance': '4', 'op_sequence': '49'},
 {'op': 'fregrid', 'op_instance': '5', 'op_sequence': '58'},
 {'op': 'fregrid', 'op_instance': '6', 'op_sequence': '67'},
 {'op': 'fregrid', 'op_instance': '7', 'op_sequence': '80'},
 {'op': 'hsmget', 'op_instance': '1', 'op_sequence': '1'},
 {'op': 'hsmget', 'op_instance': '1', 'op_sequence': '3'},
 {'op': 'hsmget', 'op_instance': '1', 'op_sequence': '5'},
 {'op': 'hsmget', 'op_instance': '1', 'op_sequence': '7'},
 

#### <a name="filter-processes">Filtering and Ordering Processes</a>

`order`, `limit` and `fltr` should be used where possible to reduce query time.

In [22]:
# now let's say we care about a particular operation. 
# Let's find the processes in the operation, and
# sort them by the cpu_time, and then see the top offenders
ncatted_procs = eq.get_procs(jobs_orm, \
                             tag = {'op': 'ncatted'}, \
                             order='desc(p.cpu_time)', \
                             limit=10, \
                             fmt='pandas')
ncatted_procs[['jobid', 'tags', 'exename', 'duration', 'cpu_time']]

,jobid,tags,exename,duration,cpu_time
0,680181,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,1256,58990
1,680181,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncdump,1112,53991
2,629337,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,1143,48992
3,693147,"{'op': 'ncatted', 'op_sequence': '41', 'op_ins...",ncatted,1118,48992
4,629337,"{'op': 'ncatted', 'op_sequence': '32', 'op_ins...",ncatted,1119,48991
5,627922,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,1037,47992
6,696127,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,1082,47992
7,633144,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,1085,47991
8,692544,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,1053,47991
9,693147,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,1042,46991


We could have used a more precise tag, such as `{'op': 'ncatted', 'op_sequence': 85}`,
for more granular selection. And, maybe an exename, such as `ncatted`.

In [23]:
procs = eq.get_procs(jobs_orm, tag='op:ncatted;op_sequence:85', \
                     fltr='p.exename == "ncatted"', \
                     order='desc(p.duration)', \
                     fmt='pandas')
procs[['jobid', 'tags', 'exename', 'duration', 'cpu_time', 'exitcode']]

,jobid,tags,exename,duration,cpu_time,exitcode
0,680181,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,1256,58990,0
1,629337,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,1143,48992,0
2,633144,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,1085,47991,0
3,696127,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,1082,47992,0
4,692544,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,1053,47991,0
5,693147,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,1042,46991,0
6,627922,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,1037,47992,0
7,804285,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,588,22995,0
8,676007,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,569,23995,0
9,802954,"{'op': 'ncatted', 'op_sequence': '85', 'op_ins...",ncatted,536,21996,0


#### <a name="thread-metrics-aggregation">Process contains aggregated thread metrics (ADVANCED TOPIC)</a>

The `pandas` (and the `dict`) formats, in addition to having process-level data in each row, also have fields that represent metrics aggregated across the underlying threads of the process, such, as
`rssmax`, `cpu_time`, and `rchar`. The ORM `Process` object instead has a `threads_sums` attribute, 
which is a dictionary containing the above fields.

In [24]:
procs.columns.values

array(['PERF_COUNT_SW_CPU_CLOCK', 'args', 'cancelled_write_bytes',
       'cpu_time', 'created_at', 'delayacct_blkio_time', 'duration',
       'end', 'exename', 'exitcode', 'gen', 'group', 'guest_time', 'host',
       'id', 'inblock', 'inclusive_cpu_time', 'invol_ctxsw', 'job',
       'jobid', 'majflt', 'minflt', 'numtids', 'outblock', 'parent',
       'path', 'pgid', 'pid', 'ppid', 'processor', 'rchar',
       'rdtsc_duration', 'read_bytes', 'rssmax', 'sid', 'start', 'syscr',
       'syscw', 'systemtime', 'tags', 'time_oncpu', 'time_waiting',
       'timeslices', 'updated_at', 'user', 'user+system', 'usertime',
       'vol_ctxsw', 'wchar', 'write_bytes'], dtype=object)

## <a name="ops">Operations</a>

An operation is simply a collection of processes that share a tag.
The collection of processes form a **forest**. In the trivial case the forest will be a single
tree if there is only one root process. 

### <a name="select-op-procs">Selecting processes in an operation</a>

We select the processes in an operation by passing a tag to `get_procs`.
You may limit the selection to a single job or multiple jobs using the
`jobs` parameter to `get_procs`.

In [25]:
# below we use the ORM format as we just want a count on the number of processes in the operation
hsmget_op_procs = eq.get_procs(jobs, tag='op:hsmget', fmt='orm')
hsmget_op_procs.count()

27720

### <a name="op-queries">Operation queries (ADVANCED TOPIC)</a>

<a name="cumulative-cpu-time-for-op"></a>Let's get the cumulative cpu time across all the processes for the operation.

In [26]:
eq.select(p.cpu_time for p in hsmget_op_procs).sum()

2540237921.0

<a name="op-top-exe"></a>
Now let us see the top executables by cpu time for the operation. This requires doing a `select` with
a group-by executable name.

In [27]:
eq.select((p.exename, sum(p.cpu_time)) for p in hsmget_op_procs).order_by('eq.desc(sum(p.cpu_time))')[:][:10]

[('globus-url-copy', 2005629923.0),
 ('perl', 252404075.0),
 ('tcsh', 78575804.0),
 ('cut', 27399825.0),
 ('host', 18195405.0),
 ('grid-proxy-info', 15352101.0),
 ('make', 15155680.0),
 ('which', 13790410.0),
 ('ncdump', 12461684.0),
 ('uberftp', 11301956.0)]

As you can see the `globus-url-copy` takes `2005` seconds, and the next program `perl` takes an order of magnitude less time.

### <a name="op-metrics">Aggregating operation metrics</a>

Writing ORM queries for aggregating operation is cumbersome, so we have an API call <a name="op-metrics">
`op_metrics`</a> to aggregate fields across processes in a given operation. In its simplest invocation 
we pass it a list of one or more jobs:

In [28]:
# widen width of column display width to show full tag
import pandas as pd
pd.set_option('display.max_colwidth', 200)

# get the operations with the top cpu_time summed across all processes. 
# Note, cpu_time is better measure of time spent in an operation than 
# 'duration', which might end up double-counting as in a 
# parent-child process scenario, where the parent waits on the time child.
ops_df = eq.op_metrics(jobs, fmt='pandas').sort_values(by='cpu_time', ascending=False)
ops_df[['jobid', 'tags', 'duration', 'cpu_time']][:10]

,jobid,tags,duration,cpu_time
407,625172,"{'op': 'hsmget', 'op_sequence': '26', 'op_instance': '8'}",3188764562,341115936
11,625172,"{'op': 'fregrid', 'op_sequence': '117', 'op_instance': '1'}",86683146,86695819
599,633144,"{'op': 'fregrid', 'op_sequence': '80', 'op_instance': '7'}",79588330,77053280
1059,627922,"{'op': 'timavg', 'op_sequence': '37', 'op_instance': '3'}",149740783,74195710
603,692544,"{'op': 'fregrid', 'op_sequence': '80', 'op_instance': '7'}",72670765,71719091
597,627922,"{'op': 'fregrid', 'op_sequence': '80', 'op_instance': '7'}",71467492,71100185
605,696127,"{'op': 'fregrid', 'op_sequence': '80', 'op_instance': '7'}",72936461,70559268
600,676007,"{'op': 'fregrid', 'op_sequence': '80', 'op_instance': '7'}",73611512,69214473
598,629337,"{'op': 'fregrid', 'op_sequence': '80', 'op_instance': '7'}",69376251,68409594
607,804285,"{'op': 'fregrid', 'op_sequence': '80', 'op_instance': '7'}",67895264,67782689


#### <a name="dm-ops">Data movement operations</a>
The above call resulted in a lot of operations. The `op_metric` call can take an optional 
list of tags if one knows the operations one cares about. Let's figure out the time spent
in data movement operations</a> v. useful work.
In the call to `op_metrics` below, we pass in the *list of tags* that
represent the data-movement operations. As it's a list of tags, it's like
an OR-operation with the tags.

In [29]:
dm_tags = ['op:hsmget', 'op:cp', 'op:dmget', 'op:gcp', 'op:mv', 'op:untar', 'op:tar', 'op:rm']
dm_ops_df = eq.op_metrics(jobs, tags = dm_tags)
dm_ops_df[['jobid', 'tags', 'cpu_time', 'duration', 'num_procs']]

,jobid,tags,cpu_time,duration,num_procs
0,625172,{'op': 'hsmget'},525588783,16145107597,8860
1,627922,{'op': 'hsmget'},151229296,7160025791,1713
2,629337,{'op': 'hsmget'},221437661,7716458753,1713
3,633144,{'op': 'hsmget'},207422750,7509555121,1713
4,676007,{'op': 'hsmget'},187083822,14508170608,1713
5,680181,{'op': 'hsmget'},230162385,7836626559,1713
6,685016,{'op': 'hsmget'},123305670,7585973173,1713
7,692544,{'op': 'hsmget'},190831238,1526509158,1713
8,693147,{'op': 'hsmget'},199442956,5148360707,1730
9,696127,{'op': 'hsmget'},201617665,4595265912,1713


While the query above helps, we would like it to aggregate across jobs by tag. This
is easily accomplished by passing the <a name="group-by-tag">`group_by_tag`</a> 
argument to `op_metrics`:

In [30]:
dm_ops_df_grouped = eq.op_metrics(jobs, tags = dm_tags, group_by_tag = True)
dm_ops_df_grouped[['tags', 'cpu_time', 'duration', 'num_procs']]

,tags,cpu_time,duration,num_procs
0,{'op': 'cp'},125672164,553131029,12827
1,{'op': 'hsmget'},2540237921,87996944703,27720
2,{'op': 'mv'},142701408,992812444,900
3,{'op': 'rm'},26662950,47021607,2940
4,{'op': 'untar'},45750643,99932219,2513


So, the total time spent in all data-movement operations can be calculated easily.

In [31]:
dm_ops_df_grouped['cpu_time'].sum()/1e6

2881.025086

In [32]:
# total time spent in the jobs
eq.select(j.cpu_time for j in jobs_orm).sum()/1e6

7351.686315

In [33]:
# data-movement as a percentage of total time
round((100*__/_), 2)

39.19

#### <a name="cpu-time-v-duration">cpu time v. duration</a>
So, the data-movement operations take about `39%` of the total cpu time across our jobs.
There is a reason we did not use `duration` for our calculation, and instead we used
`cpu_time` (a.k.a exclusive cpu time). The reason is that `duration` can get counted multiple
times if a process forks and waits for a child to terminate. The `duration` or `wall-clock` 
time will end up getting calculated both for the parent process and the child process. 
`cpu_time` on the other hand is the actual time spent on the cpu, and cannot be counted twice 
in such a scenario.

## <a name="thread-query">Thread Query</a>

The thread query requires passing one or more *process primary keys* or `Process`
objects to `get_thread_metrics`. Let's illustrate this with an example, where
we first obtain the <a name="root-process">root process</a> of a job:

In [34]:
# let's find the root process for a particular job
root = eq.root('629337', fmt='terse')
root

1

In [35]:
root_threads_df = eq.get_thread_metrics(root)
display(root_threads_df.columns.values)
root_threads_df[['process_pk', 'tid', 'usertime', 'systemtime', 'rssmax']]

array(['tid', 'start', 'end', 'usertime', 'systemtime', 'rssmax',
       'minflt', 'majflt', 'inblock', 'outblock', 'vol_ctxsw',
       'invol_ctxsw', 'num_threads', 'starttime', 'processor',
       'delayacct_blkio_time', 'guest_time', 'rchar', 'wchar', 'syscr',
       'syscw', 'read_bytes', 'write_bytes', 'cancelled_write_bytes',
       'time_oncpu', 'time_waiting', 'timeslices', 'rdtsc_duration',
       'PERF_COUNT_SW_CPU_CLOCK', 'process_pk'], dtype=object)

,process_pk,tid,usertime,systemtime,rssmax
0,1,16269,454930,352946,5516


## <a name="useful-attributes">Useful attributes in Job, Process and Thread objects</a>

The following are some useful attributes of the job, process and thread objects.
They are accessible when using the `orm` format. They are available in the 
`pandas` and `dict` formats. There is one important thing to note:

`proc_sums` field of the Job object is masked for `pandas` and `dict` formats
and the underlying keys of the dictionary are exposed at the top-level.

`threads_sums` field of the Process object is masked for `pandas` and `dict` format
and the underlying keys of the dictionary are exposed at the top-level.

### Job Attributes
 - duration: this is the wallclock time in microseconds
 - cpu_time: user+system time aggregated across all processes of the job
 - start:    start time in microseconds since epoch
 - end:      end time in microseconds since epoch
 - jobid:    database id for job (unique)
 - exitcode: return code from job
 - tags:     dict of key/value pairs
 - processes:list of processes belonging to job
 - proc_sums: aggregates across processes of a job
 

### Process Attributes
 - duration: this is the wallclock time in microseconds
 - cpu_time: exclusive user+system time for process (aggregated across it's threads)
 - inclusive_cpu_time: user+system time for the process and *all its descendants*
 - start:    start time in microseconds since epoch
 - end:      end time in microseconds since epoch
 - tags:     dict of key/value pairs
 - threads_df: json serialized dataframe of process threads (ADVANCED)
 - threads_sums: key/value pairs consisting of sums of thread metrics (ADVANCED)
 - numtids:  number of threads
 - exename
 - args
 - pid
 - ppid
 - id:       database ID for process
 - exitcode
 - parent
 - children
 - ancestors
 - descendants
 
 
### Thread Attributes
 - usertime
 - systemtime
 - user+system
 - rssmax
 - majflt
 - read_bytes
 - write_bytes

## <a name="useful-queries">Misc. queries</a>

Below we have some more queries to give you a flavor of how to use the API

In [36]:
# ordinarily we would first find the job and then probe downwards
# You can use tags or fltr arguments to find the job
# As we did not include job tags in this script, let's just find the job using
# its job id
job = eq.get_jobs('676007', fmt='dict')[0]
job

{'PERF_COUNT_SW_CPU_CLOCK': 605349533801,
 'account': None,
 'all_proc_tags': [{'op': 'cp', 'op_instance': '11', 'op_sequence': '66'},
  {'op': 'cp', 'op_instance': '15', 'op_sequence': '79'},
  {'op': 'cp', 'op_instance': '3', 'op_sequence': '30'},
  {'op': 'cp', 'op_instance': '5', 'op_sequence': '39'},
  {'op': 'cp', 'op_instance': '7', 'op_sequence': '48'},
  {'op': 'cp', 'op_instance': '9', 'op_sequence': '57'},
  {'op': 'dmput', 'op_instance': '2', 'op_sequence': '89'},
  {'op': 'fregrid', 'op_instance': '2', 'op_sequence': '31'},
  {'op': 'fregrid', 'op_instance': '3', 'op_sequence': '40'},
  {'op': 'fregrid', 'op_instance': '4', 'op_sequence': '49'},
  {'op': 'fregrid', 'op_instance': '5', 'op_sequence': '58'},
  {'op': 'fregrid', 'op_instance': '6', 'op_sequence': '67'},
  {'op': 'fregrid', 'op_instance': '7', 'op_sequence': '80'},
  {'op': 'hsmget', 'op_instance': '1', 'op_sequence': '1'},
  {'op': 'hsmget', 'op_instance': '1', 'op_sequence': '3'},
  {'op': 'hsmget', 'op_inst

In [37]:
# now get the processes that are part of this job, let's sort them by the inclusive time
# we need to pass in the job id to restrict the query to a particular job
# the inclusive_cpu_time sums all the cpu times of the process and its dependents
# in this case you can see that after the top-level 'bash', the 'find' with the
# -exec stat shows up
procs = eq.get_procs('676007', order = 'desc(p.inclusive_cpu_time)', fmt='pandas', limit=10)
procs[['exename', 'duration', 'inclusive_cpu_time', 'exitcode']]

,exename,duration,inclusive_cpu_time,exitcode
0,tcsh,1.008058e+10,607624279,0
1,fregrid,7.261159e+07,68253623,0
2,ncra,8.840326e+07,55002636,0
3,tcsh,4.076242e+07,38443149,0
4,TAVG.exe,4.035406e+07,38386164,0
5,tcsh,3.485533e+07,34631728,0
6,TAVG.exe,3.459367e+07,34583741,0
7,perl,3.851296e+07,32827920,0
8,perl,3.796058e+07,32017044,0
9,make,3.366503e+07,31420174,0


<a name="process-tree-walk"></a>Let's do a walk through the process tree.

In [38]:
# now let's walk through the process tree. To make this easy, we use the 'orm' format
# let's sort the processes by exclusive cpu time
# You will get a sorted list of ORM objects, let's see the top 10
procs = eq.get_procs('676007', order = 'desc(p.cpu_time)', fmt='orm')[:10]
procs

[Process[32878], Process[32683], Process[31393], Process[31890], Process[32139], Process[29747], Process[32388], Process[30508], Process[30323], Process[31642]]

In [39]:
# lets pick up the first
p = procs[0]
p

Process[32878]

In [40]:
p.exename

'fregrid'

In [41]:
p.exename, p.args, p.duration, len(p.children), p.numtids

('fregrid',
 '--standard_dimension --input_mosaic ocean_mosaic.nc --input_file all --interp_method conserve_order1 --remap_file .fregrid_remap_file_360_by_180.nc --nlon 360 --nlat 180 --scalar_field volcello,thkcello,vo,vmo,vhGM,vhml --output_file out.nc',
 72611586.0,
 0,
 1)

In [42]:
parent = p.parent
parent

Process[29592]

In [43]:
parent.exename, parent.args, parent.pid, len(parent.children), len(parent.descendants)

('tcsh',
 '-f /home/Jeffrey.Durachta/ESM4/DECK/ESM4_historical_D151/gfdl.ncrc4-intel16-prod-openmp/scripts/postProcess/ESM4_historical_D151_ocean_month_rho2_1x1deg_18740101.tags',
 27339,
 729,
 3309)

In [44]:
# let's see the aggregate thread metrics for this process
p.threads_sums

{'PERF_COUNT_SW_CPU_CLOCK': 68221670024,
 'cancelled_write_bytes': 0,
 'delayacct_blkio_time': 0,
 'guest_time': 0,
 'inblock': 12968064,
 'invol_ctxsw': 180,
 'majflt': 3,
 'minflt': 9946,
 'outblock': 2141984,
 'processor': 0,
 'rchar': 15140041925,
 'rdtsc_duration': 251077780608,
 'read_bytes': 6639648768,
 'rssmax': 58112,
 'syscr': 1741380,
 'syscw': 33346,
 'systemtime': 5995088,
 'time_oncpu': 68265620909,
 'time_waiting': 15819937,
 'timeslices': 536,
 'user+system': 68253623,
 'usertime': 62258535,
 'vol_ctxsw': 355,
 'wchar': 2185067697,
 'write_bytes': 1096695808}

In [45]:
# let's get the thread dataframes for p
eq.get_thread_metrics(p)

,tid,start,end,usertime,systemtime,rssmax,minflt,majflt,inblock,outblock,...,syscw,read_bytes,write_bytes,cancelled_write_bytes,time_oncpu,time_waiting,timeslices,rdtsc_duration,PERF_COUNT_SW_CPU_CLOCK,process_pk
0,5488,1560525416253004,1560525488864590,62258535,5995088,58112,9946,3,12968064,2141984,...,33346,6639648768,1096695808,0,68265620909,15819937,536,251077780608,68221670024,32878


In [46]:
# Let's explore a particular operation in a job, and see which processes took the 
# top *inclusive* cpu time.
# Let's limit the output to the top 5 results
# and let's get a pandas dataframe
procs = eq.get_procs(jobs, tag = 'op_sequence:159', order='desc(p.inclusive_cpu_time)', limit=5, fmt='pandas')
procs[['exename', 'args', 'cpu_time', 'inclusive_cpu_time', 'duration']]

,exename,args,cpu_time,inclusive_cpu_time,duration
0,fregrid,--standard_dimension --input_mosaic ocean_mosaic.nc --input_file annual --interp_method conserve_order1 --remap_file .fregrid_remap_file_360_by_180.nc --nlon 360 --nlat 180 --scalar_field volcello...,10237442,10237442,10219947
1,mv,out.nc annual.nc,462929,462929,456714
2,mv,annual.nc ocean_month_rho2_1x1deg.1851.ann.nc,43992,43992,36877


<a name="failed-procs"></a>Let's see if there are any failed processes in our job selection.

In [47]:
# Let's find the failed processes across our jobs subset
failed_procs = eq.get_procs(jobs_orm, fltr='p.exitcode > 0', fmt='pandas')
failed_procs[['jobid', 'exename', 'args', 'exitcode', 'tags']]

,jobid,exename,args,exitcode,tags
0,625172,ln,-f /ptmp/Jeffrey.Durachta/archive/Jeffrey.Durachta/ESM4/DECK/ESM4_historical_D151/gfdl.ncrc4-intel16-prod-openmp/history/18500101.nc/18500101.ocean_month_rho2.nc /vftmp/Jeffrey.Durachta/job625172/...,1,"{'op': 'hsmget', 'op_sequence': '1', 'op_instance': '1'}"
1,625172,which,globus-ftp-client-cksm-test,1,"{'op': 'hsmget', 'op_sequence': '1', 'op_instance': '1'}"
2,625172,which,globus-ftp-client-mlst-test,1,"{'op': 'hsmget', 'op_sequence': '1', 'op_instance': '1'}"
3,625172,which,globus-ftp-client-ascii-verbose-list-test,1,"{'op': 'hsmget', 'op_sequence': '1', 'op_instance': '1'}"
4,625172,which,globus-ftp-client-delete-test,1,"{'op': 'hsmget', 'op_sequence': '1', 'op_instance': '1'}"
5,625172,grep,_DeflateLevel,1,"{'op': 'hsmget', 'op_sequence': '1', 'op_instance': '1'}"
6,625172,grep,_Shuffle,1,"{'op': 'hsmget', 'op_sequence': '1', 'op_instance': '1'}"
7,625172,ln,-f /ptmp/Jeffrey.Durachta/archive/Jeffrey.Durachta/ESM4/DECK/ESM4_historical_D151/gfdl.ncrc4-intel16-prod-openmp/history/18500101.nc/18500101.ocean_static.nc /vftmp/Jeffrey.Durachta/job625172/ESM4...,1,"{'op': 'hsmget', 'op_sequence': '2', 'op_instance': '3'}"
8,625172,which,globus-ftp-client-cksm-test,1,"{'op': 'hsmget', 'op_sequence': '2', 'op_instance': '3'}"
9,625172,which,globus-ftp-client-mlst-test,1,"{'op': 'hsmget', 'op_sequence': '2', 'op_instance': '3'}"


Let's focus only on a particular operation, and prune the list a bit

In [48]:
failed_procs = eq.get_procs(jobs, tag='op_sequence:79', fltr='p.exitcode > 0', fmt='pandas')
failed_procs[['jobid', 'id', 'exename', 'args', 'exitcode']]

,jobid,id,exename,args,exitcode
0,627922,22480,which,globus-ftp-client-cksm-test,1
1,627922,22481,which,globus-ftp-client-mlst-test,1
2,627922,22482,which,globus-ftp-client-ascii-verbose-list-test,1
3,627922,22483,which,globus-ftp-client-delete-test,1
4,629337,3125,which,globus-ftp-client-cksm-test,1
5,629337,3126,which,globus-ftp-client-mlst-test,1
6,629337,3127,which,globus-ftp-client-ascii-verbose-list-test,1
7,629337,3128,which,globus-ftp-client-delete-test,1
8,633144,29317,which,globus-ftp-client-cksm-test,1
9,633144,29318,which,globus-ftp-client-mlst-test,1


In [49]:
# let's explore one of the failed processes
p = eq.Process[int(failed_procs.loc[0,'id'])]
p.exename, p.exitcode, p.args, p.duration, p.parent

('which', 1, 'globus-ftp-client-cksm-test', 2963.0000000000005, Process[22467])

## <a name="linux-kernel">Case Study - Linux Kernel Compile</a>

Start by importing the data for this experiment (import takes around half an hour on my laptop):
```
$ ./epmt -v submit test/data/outliers/*.tgz
```

In [50]:
# start by locating the job in the database using tags
# you can specify tags as a dict or a string
# use fmt='terse' as we just want to know the job id
kern_jobs = j = eq.get_jobs(tag = 'exp_name:linux_kernel', fmt='terse')
j

['kern-6656-20190614-194024',
 'kern-6656-20190614-192044-outlier',
 'kern-6656-20190614-191138',
 'kern-6656-20190614-190245']

In [51]:
# let's get all the tags associated with the processes of the jobs
eq.job_proc_tags('kern-6656-20190614-194024')

[{'op': 'build', 'op_instance': '4', 'op_sequence': '4'},
 {'op': 'clean', 'op_instance': '5', 'op_sequence': '5'},
 {'op': 'configure', 'op_instance': '3', 'op_sequence': '3'},
 {'op': 'download', 'op_instance': '1', 'op_sequence': '1'},
 {'op': 'extract', 'op_instance': '2', 'op_sequence': '2'}]

In [52]:
# let's see the processes in the download phase
download = eq.get_procs('kern-6656-20190614-194024', tag = 'op:download', fmt='pandas')
download[['exename', 'args', 'exitcode', 'duration']]

,exename,args,exitcode,duration
0,wget,https://cdn.kernel.org/pub/linux/kernel/v5.x/linux-5.1.7.tar.xz,0,8795277


In [53]:
# So, there was only the single program wget and the duration shows the wallclock time
# Now let's sudy the configre phase. We expect it to have many processes. Whenever the 
# number of processes is large, it is a good idea to use order_by and limit, particularly
# if the format is dict or pandas. The 'orm' and 'terse' formats are usually fast already.
configure_procs = eq.get_procs('kern-6656-20190614-194024', tag = 'op:configure', fmt='orm')
configure_procs.count()

1044

In [54]:
# As you can see, that a lot of processes. Let's use order and limit to get a better understanding
# So, we will re-run the query but this time, we will sort by inclusive_cpu_time and get the top 10 processes
configure = eq.get_procs('kern-6656-20190614-194024', tag = 'op:configure', order = 'desc(p.inclusive_cpu_time)', limit = 10, fmt='pandas')
configure[['exename', 'args', 'pid', 'duration', 'inclusive_cpu_time', 'cpu_time']]

,exename,args,pid,duration,inclusive_cpu_time,cpu_time
0,make,tinyconfig,20637,31056851,20718776,35155
1,make,-f ./scripts/Makefile.build obj=scripts/kconfig tinyconfig,20989,26143551,18082899,17830
2,make,-f ./Makefile allnoconfig tiny.config,20990,26123688,18065069,15000
3,dash,-c set -e; \nfor i in allnoconfig tiny.config; do \ntmake -f ./Makefile $i; \ndone,20991,26086193,18050069,13968
4,make,-f ./Makefile allnoconfig,20992,15021091,10086993,37768
5,make,-f ./Makefile tiny.config,21502,11027793,7949108,38075
6,make,-f ./scripts/Makefile.build obj=scripts/kconfig allnoconfig,21224,11213610,7856192,28035
7,make,-f ./scripts/Makefile.build obj=scripts/kconfig tiny.config,21767,7644974,5724650,26152
8,dash,"-c yes """" | make -f ./Makefile oldconfig",21849,5595704,4344229,13160
9,make,-f ./Makefile oldconfig,21851,5573469,4312809,43587


In [55]:
# total time for operation
c = eq.get_procs('kern-6656-20190614-194024', tag = 'op:configure', fmt='orm')
eq.select(p.cpu_time for p in c).sum()

20718776.0

In [56]:
# another trick that works to get the max time for an operation is
# to find the process with the max value for duration. This works if
# you have a top-level process that spawned the rest
# Notice we use order and limit
root_build_proc = eq.get_procs('kern-6656-20190614-194024', tag = 'op:build', order='desc(p.duration)', limit=5, fmt='pandas')
root_build_proc[['exename', 'args', 'duration', 'inclusive_cpu_time', 'exitcode']]

,exename,args,duration,inclusive_cpu_time,exitcode
0,make,,458045491,381227732,0
1,make,-f ./scripts/Makefile.build obj=arch/x86 need-builtin=1,103167887,81574246,0
2,make,-f ./scripts/Makefile.build obj=kernel need-builtin=1,102770574,81903208,0
3,make,-f ./scripts/Makefile.build obj=arch/x86/kernel need-builtin=1,57300252,47245312,0
4,make,-f ./scripts/Makefile.build obj=lib need-builtin=1,55106357,40724592,0


In [57]:
# Above, you notice the build operation's root process 'make' took
# a long time 

# Now let's see if any process failed in the build phase
# If you use 'orm' you get access to 'count', which is superfast as it
# uses sql to a count directly rather than load all the fields of the matching processes
eq.get_procs('kern-6656-20190614-194024', tag = 'op:build', fltr='p.exitcode != 0', fmt='orm').count()

101

### <a name="timeline">Timeline</a>
Sometimes you want to get a timeline of the processes in the order they were spawned

In [58]:
procs = eq.timeline('kern-6656-20190614-194024', fmt='pandas', limit=25)
procs[['exename', 'tags', 'start', 'duration']]

,exename,tags,start,duration
0,tempfile,None,2019-06-14 14:10:25.072015,408
1,rm,None,2019-06-14 14:10:25.102954,391
2,mkdir,None,2019-06-14 14:10:25.113562,330
3,wget,"{'op': 'download', 'op_sequence': '1', 'op_instance': '1'}",2019-06-14 14:10:25.126758,8795277
4,tar,"{'op': 'extract', 'op_sequence': '2', 'op_instance': '2'}",2019-06-14 14:10:33.952468,50762305
5,xz,"{'op': 'extract', 'op_sequence': '2', 'op_instance': '2'}",2019-06-14 14:10:33.966028,50734713
6,make,"{'op': 'configure', 'op_sequence': '3', 'op_instance': '3'}",2019-06-14 14:11:24.750256,31056851
7,dash,"{'op': 'configure', 'op_sequence': '3', 'op_instance': '3'}",2019-06-14 14:11:24.767841,47419
8,sed,"{'op': 'configure', 'op_sequence': '3', 'op_instance': '3'}",2019-06-14 14:11:24.781268,32116
9,uname,"{'op': 'configure', 'op_sequence': '3', 'op_instance': '3'}",2019-06-14 14:11:24.810794,278


In [59]:
# The orm also gives an easy way to navigate the process hierarchy
# Let's use the ORM directly to walk through the job
j = eq.get_jobs('kern-6656-20190614-194024', fmt='orm').first()
j

Job['kern-6656-20190614-194024']

In [60]:
# Notice we have a Job object. The processes in the job
# are available as j.processes
j.duration, j.cpu_time, j.exitcode, j.tags

(565343997.0,
 450359378.0,
 0,
 {'exp_component': 'kernel_tiny',
  'exp_name': 'linux_kernel',
  'launch_id': '6656',
  'seqno': '6'})

In [61]:
# let's see the process that took the max cpu time
max_cpu_proc = j.processes.order_by('desc(p.cpu_time)').limit(1)[0]
max_cpu_proc.exename, max_cpu_proc.pid, max_cpu_proc.cpu_time, max_cpu_proc.duration

('xz', 18584, 24339158.0, 50734713.00000001)

In [62]:
# let's get details on the build operation
b = eq.get_procs(j, tag = 'op:build', order='desc(p.inclusive_cpu_time)', fmt='orm')
b

In [63]:
# Above we get a Query object, we can iterate over it, convert
# it to a list or get a slice of it
b_limit = b.order_by('eq.desc(p.inclusive_cpu_time)').limit(5)
b_limit

[Process[81881], Process[81908], Process[81887], Process[81888], Process[81906]]

In [64]:
# observe that we don't actually do any queries until we start using
# the result
top_cpu = b_limit[0]
top_cpu

Process[81881]

In [65]:
top_cpu.exename, top_cpu.args, top_cpu.duration, top_cpu.cpu_time

('make', '', 458045491.0, 206921.0)

In [66]:
# now we get access to the parent/children/ancestors/descendats of this process
max(top_cpu.descendants.exitcode)

128

In [67]:
# so one or more descendant processes failed, let's find which ones
failed = top_cpu.descendants.filter('p.exitcode != 0')
failed.count()

101

In [68]:
# we can convert a Query object to a pandas dataframe anytime
# Note, you can also pass a Query object to eq.get_procs and achive format 
# conversion. 
import pandas as pd
df = pd.DataFrame([p.to_dict() for p in failed])
df[['exename', 'args', 'start', 'end', 'pid', 'ppid', 'exitcode']]

,exename,args,start,end,pid,ppid,exitcode
0,gcc-5,-Werror -D__KERNEL__ -Wall -Wundef -Werror=strict-prototypes -Wno-trigraphs -fno-strict-aliasing -fno-common -fshort-wchar -fno-PIE -Werror=implicit-function-declaration -Werror=implicit-int -Wno-...,2019-06-14 14:17:44.460188,2019-06-14 14:17:44.462283,1708,1707,1
1,gcc-5,-Werror -fsanitize-coverage=trace-pc -E -x c /dev/null -o /dev/null,2019-06-14 14:12:05.472325,2019-06-14 14:12:05.473403,22954,22953,1
2,dash,-c gcc --version 2>&1 | head -n 1 | grep clang,2019-06-14 14:11:56.013604,2019-06-14 14:11:56.035513,22301,22290,1
3,dash,-c cat include/config/kernel.release 2> /dev/null,2019-06-14 14:12:24.159600,2019-06-14 14:12:24.174031,24812,22290,1
4,grep,-qF .rel.local,2019-06-14 14:19:38.257873,2019-06-14 14:19:38.260505,6125,6113,1
5,gcc-5,-Werror -D__KERNEL__ -m32 -O2 -fno-strict-aliasing -fPIE -DDISABLE_BRANCH_PROFILING -march=i386 -mno-mmx -mno-sse -ffreestanding -fno-stack-protector -Waddress-of-packed-member -c -x c /dev/null -...,2019-06-14 14:19:31.672761,2019-06-14 14:19:31.674072,5908,5907,1
6,gcc-5,-Werror -D__KERNEL__ -Wall -Wundef -Werror=strict-prototypes -Wno-trigraphs -fno-strict-aliasing -fno-common -fshort-wchar -fno-PIE -Werror=implicit-function-declaration -Werror=implicit-int -Wno-...,2019-06-14 14:12:00.899560,2019-06-14 14:12:00.901039,22610,22609,1
7,dash,-c cat include/config/kernel.release 2> /dev/null,2019-06-14 14:12:22.636773,2019-06-14 14:12:22.651115,24719,22290,1
8,dash,-c gcc --version 2>&1 | head -n 1 | grep clang,2019-06-14 14:12:05.931282,2019-06-14 14:12:05.955433,22967,22290,1
9,grep,clang,2019-06-14 14:12:05.944762,2019-06-14 14:12:05.953573,22970,22967,1


In [69]:
# first we ask for the aggregate metrics for single job
# Here, we don't specify any tags. For single jobs, when
# we don't specify the operation/tags, they are queried from the job
op_sums = eq.op_metrics(jobs='kern-6656-20190614-194024', fmt='pandas')
display(op_sums.columns.values)
op_sums[['jobid', 'tags', 'duration', 'cpu_time']]

array(['PERF_COUNT_SW_CPU_CLOCK', 'cancelled_write_bytes', 'cpu_time',
       'delayacct_blkio_time', 'duration', 'guest_time', 'inblock',
       'invol_ctxsw', 'job', 'jobid', 'majflt', 'minflt', 'num_procs',
       'numtids', 'outblock', 'processor', 'rchar', 'rdtsc_duration',
       'read_bytes', 'rssmax', 'syscr', 'syscw', 'systemtime', 'tags',
       'time_oncpu', 'time_waiting', 'timeslices', 'user+system',
       'usertime', 'vol_ctxsw', 'wchar', 'write_bytes'], dtype=object)

,jobid,tags,duration,cpu_time
0,kern-6656-20190614-194024,"{'op': 'build', 'op_sequence': '4', 'op_instance': '4'}",2253935203,381227732
1,kern-6656-20190614-194024,"{'op': 'clean', 'op_sequence': '5', 'op_instance': '5'}",6340381,5561711
2,kern-6656-20190614-194024,"{'op': 'configure', 'op_sequence': '3', 'op_instance': '3'}",236011451,20718776
3,kern-6656-20190614-194024,"{'op': 'download', 'op_sequence': '1', 'op_instance': '1'}",8795277,1899179
4,kern-6656-20190614-194024,"{'op': 'extract', 'op_sequence': '2', 'op_instance': '2'}",101497018,40930869


In [70]:
# Now let's run the same query against all the kernel build jobs. In this case, we need
# to provide a list of tags (or a single tag) for the operation
eq.op_metrics(kern_jobs, tags=['op:build', 'op:configure'])[['job','tags', 'cpu_time','num_procs', 'rssmax']]

,job,tags,cpu_time,num_procs,rssmax
0,kern-6656-20190614-190245,{'op': 'build'},380807266,9549,71905040
1,kern-6656-20190614-191138,{'op': 'build'},381619141,9549,72017388
2,kern-6656-20190614-192044-outlier,{'op': 'build'},540737924,9549,72094180
3,kern-6656-20190614-194024,{'op': 'build'},381227732,9549,71911808
4,kern-6656-20190614-190245,{'op': 'configure'},20735346,1044,7323148
5,kern-6656-20190614-191138,{'op': 'configure'},20476970,1044,7315736
6,kern-6656-20190614-192044-outlier,{'op': 'configure'},27253558,1044,7324964
7,kern-6656-20190614-194024,{'op': 'configure'},20718776,1044,7320600


In [71]:
# let's look at a particular job and see the processes with largest page faults
# across all threads for only the build operation
df = eq.get_procs('kern-6656-20190614-194024', tag='op:build', order='desc(p.threads_sums["majflt"])', limit=5, fmt='pandas')
df[['exename', 'args', 'majflt', 'cpu_time']]

,exename,args,majflt,cpu_time
0,modpost,-o ./Module.symvers -E vmlinux.o,345,41604
1,build,arch/x86/boot/setup.bin arch/x86/boot/vmlinux.bin arch/x86/boot/zoffset.h arch/x86/boot/bzImage,104,13834
2,sortextable,vmlinux,70,13134
3,vdso2c,arch/x86/entry/vdso/vdso32.so.dbg arch/x86/entry/vdso/vdso32.so arch/x86/entry/vdso/vdso-image-32.c,6,11982
4,relocs,--realmode arch/x86/realmode/rm/realmode.elf,5,12302


In [72]:
# As you may know for outlier detection we can only compare jobs with the
# same exp_name and exp_component. Let's do a query to count the number of jobs
# for each exp_component:
# For this we will use advanced ORM methods
q = eq.select((eq.count(j), j.tags['exp_component']) for j in eq.Job)
list(q[:])

[(4, 'kernel_tiny'), (12, 'ocean_month_rho2_1x1deg')]

In [73]:
# below we filter those processes of the job that exceed a certain
# wallclock time, and then sort them by the exclusive cpu time (user+system)
# fltr can be a lamdba function or a string
# limit can be useful to restrict the number of elements in the output
eq.get_procs(jobs_orm, fltr = lambda p: p.duration > 100000, order = 'desc(p.cpu_time)', limit=5, fmt='pandas')

,PERF_COUNT_SW_CPU_CLOCK,args,cancelled_write_bytes,cpu_time,created_at,delayacct_blkio_time,duration,end,exename,exitcode,...,time_oncpu,time_waiting,timeslices,updated_at,user,user+system,usertime,vol_ctxsw,wchar,write_bytes
0,94954852385,-rst -bs 6M -tcp-bs 6M -p 1 -fast -pp -g2 -dcsafe -cc 4 -vb -df /tmp/.gcp_GUC_cexf,4096,95002557,2019-08-05 07:08:44.288981,0,98113583,2019-06-10 00:31:48.307117,globus-url-copy,0,...,95017579711,84901580,5604,2019-08-05 07:08:44.288984,Jeffrey.Durachta,95002557,83149359,5346,19508489955,19508482048
1,86678363632,--standard_dimension --input_mosaic ocean_mosaic.nc --input_file 18540101.ocean_static --interp_method conserve_order1 --remap_file .fregrid_remap_file_360_by_180.nc --nlon 360 --nlat 180 --scalar...,0,86684821,2019-08-05 07:09:03.785553,0,86682157,2019-06-10 02:17:43.766677,fregrid,0,...,86698346888,2971009,204,2019-08-05 07:09:03.785556,Jeffrey.Durachta,86684821,86619831,15,48051700,20938752
2,75458262095,"--standard_dimension --input_mosaic ocean_mosaic.nc --input_file all --interp_method conserve_order1 --remap_file .fregrid_remap_file_360_by_180.nc --nlon 360 --nlat 180 --scalar_field volcello,th...",0,75468526,2019-08-05 07:09:55.287817,0,78001390,2019-06-10 22:38:59.375103,fregrid,0,...,75480854613,113414030,1424,2019-08-05 07:09:55.287820,Jeffrey.Durachta,75468526,68477589,252,2185067697,1107968000
3,74151587122,,0,74112732,2019-08-05 07:09:32.070163,0,74295537,2019-06-10 12:05:23.841926,TAVG.exe,0,...,74124652978,58536659,248,2019-08-05 07:09:32.070166,Jeffrey.Durachta,74112732,20839831,87,696904680,1146953728
4,69628474257,"--standard_dimension --input_mosaic ocean_mosaic.nc --input_file all --interp_method conserve_order1 --remap_file .fregrid_remap_file_360_by_180.nc --nlon 360 --nlat 180 --scalar_field volcello,th...",0,69644411,2019-08-05 07:10:26.841312,0,70597962,2019-06-16 18:05:47.222094,fregrid,0,...,69655757995,6463223,340,2019-08-05 07:10:26.841314,Jeffrey.Durachta,69644411,64392210,161,2185067697,1118248960
